# Cinetel Data Cleaning

Cinetel data scraped daily from their online boxoffice at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")

---

## Cleaning the df

In [ ]:
df1 = df1.drop_duplicates()

In [ ]:
df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

In [ ]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance',  '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [ ]:
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [ ]:
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [ ]:
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [ ]:
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [ ]:
from datetime import timedelta, datetime

In [ ]:
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], format='%d/%m/%Y')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

In [ ]:
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-9])

In [ ]:
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [ ]:
df1['day_of_week'] = df1['date'].dt.day_name()

In [ ]:
df1

---

## Visualizations

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt